In [360]:
.ORIG x3000

    BRnzp ASCII_CONVERT
    
;; ------------------------------------------------------------------
;; Setup for the stack 
Top:    .FILL Stack
Stack:  .BLKW #100

PUSH:   LD R5, Top
        STR R0, R5, #0      ;; Save R0 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R1, R5, #0      ;; Save R1 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R2, R5, #0      ;; Save R2 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R3, R5, #0      ;; Save R3 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R4, R5, #0      ;; Save R4 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R6, R5, #0      ;; Save R6 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R7, R5, #0      ;; Save R7 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        ST R5, Top
        JMP R6
        
POP:    LD R5, Top
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R7, R5, #0      ;; Load stack into R7
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R6, R5, #0      ;; Load Stack into R6
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R4, R5, #0      ;; Load Stack into R4
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R3, R5, #0      ;; Load Stack into R3
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R2, R5, #0      ;; Load Stack into R2
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R1, R5, #0      ;; Load Stack into R1
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R0, R5, #0      ;; Load Stack into R0
        ST R5, Top
        RET

;; ------------------------------------------------------------------

;; Subroutine to convert binary value into ASCII decimal and print.

ASCII_CONVERT:
    LD R0, CONVERT_VAL
    ST R0, TEMPX
    ST R0, TEMPY
    
ASCII_BASE: ;; Base case: if val < #10, add x30 and print

    ;; reset TEMPZ to #-1
    LD R1, NEG_ONE
    ST R1, TEMPZ
    
    ;; check val < #10
    ADD R1, R0, #-10
    BRzp ASCII_LOOP ;; if val >= #10, skip to main loop
    
    ;; convert R1 to ASCII and print
    LD R1, ASCII_POS
    ADD R0, R0, R1
    OUT R0
    
    HALT
    
ASCII_LOOP: ;; Non-base case
    
    ;; TEMPZ = z*10
    LD R0, TEMPZ ;; R0 <- z
    LD R1, TEN ;; R1 <- 10
    LEA R6, Multiply
    LD R2, PUSH_IND1
    JSRR R2         ; R0 * R1
    LD R2, MULT_PRODUCT  ; R2 <- R0 * R1   = z*10
    ST R2, TEMPZ
    
    ;; R2 = y/10
    LD R0, TEMPY ;; R0 <- y
    LEA R6, Divide
    LD R2, PUSH_IND1
    JSRR R2         ; R0 / R1
    LD R2, QUOTIENT  ; R2 <- R0 / R1       = y/10
    
    BRp ASCII_LOOP ;; if quotient is positive, repeat loop
    
    ;; else if quotient is zero, use previous quotient (current val of TEMPY) as y
    
    ;; z = z/10 since the loop has to go one extra time to hit zero
    LD R0, TEMPZ ;; R0 <- z
    LEA R6, Divide
    LD R2, PUSH_IND1
    JSRR R2         ; R0 / R1
    LD R2, QUOTIENT  ; R2 <- R0 / R1    = z/10
    
    ;; R2 = y*z
    LD R1, TEMPY ;; R1 <- y
    LEA R6, Multiply
    LD R2, PUSH_IND1
    JSRR R2         ; R0 * R1
    LD R2, MULT_PRODUCT  ; R2 <- R0 * R1     = y*z
    
    LD R3, ASCII_POS
    ADD R1, R1, R3 ;; convert y to ASCII
    OUT R1
    
    LD R0, TEMPX
    ADD R0, R0, R2 ;; x-z: current state of val - (10^n * y)
    ST R0, TEMPX
    ST R0, TEMPY
    
    BRnzp ASCII_BASE ;; return to ASCII_BASE to check for base case
    
    
ASCII_POS:   .FILL x30
CONVERT_VAL:   .FILL #12 ;; value to be converted
NEG_ONE:   .FILL #-1
TEN:   .FILL #10
TEMPX:   .FILL #0 ;; stores present state of value to be converted, after subtraction
TEMPY:   .FILL #0 ;; stores present state of value to be converted, after division
TEMPZ:   .FILL #-1 ;; stores (negative) exponent of 10

;;-----------------------------------------------------------------
;; Signposts to PUSH and POP

PUSH_IND1: .FILL PUSH
POP_IND1: .FILL POP

;;; -----------------------------------------------------
;;; Mult_Product <- R0 * R1
;;; Also uses R3 to store SIGN
Multiply: AND R2,R2,#0
  AND R3,R3,#0
  ADD R0,R0,#0         ; compare R0
  BRn MultNEG1
  BR  MultCont
MultNEG1: NOT R3,R3         ; flip SIGN
  NOT R0,R0
  ADD R0,R0,#1
MultCONT: ADD R1,R1,#0         ; compare R1
  BRn MultNEG2
  BR MultInit
MultNEG2: NOT R3,R3         ; flip SIGN
  NOT R1,R1
  ADD R1,R1,#1
MultInit: ADD R0,R0,#0      ; have R0 set the condition codes
MultLoop: BRz MultDone
  ADD R2,R2,R1
  ADD R0,R0,#-1
  BR MultLoop
MultDone: ADD R0,R3,#0
  BRzp MultRet
  NOT R2,R2
  ADD R2,R2,#1
MultRet:
  ST R2, MULT_PRODUCT           ; Product has the product
  LD R0, POP_IND1
  JMP R0
  
MULT_PRODUCT: .FILL #0

;;; -----------------------------------------------------
;;; R0 / R1
;;; Also uses R3 to store SIGN
;;;           R4 to store -R1
;;;           R5 is QUOTIENT
;;;           R6 is REMAINDER
;;;           R2 temp
Divide:   AND R3,R3,#0
  ST R3, QUOTIENT
  ST R3, REMAINDER
  ADD R0,R0,#0         ; compare R0
  BRn DivNEG1
  BR  DivCont
DivNEG1:  NOT R3,R3         ; flip SIGN
  NOT R0,R0
  ADD R0,R0,#1
DivCONT:  ADD R1,R1,#0         ; compare R1
  BRn DivNEG2
  BR DivInit
DivNEG2:  NOT R3,R3         ; flip SIGN
  NOT R1,R1
  ADD R1,R1,#1
DivInit:  ADD R4,R1,#0
  NOT R4,R4
  ADD R4,R4,#1
DivLoop:  ADD R2,R0,R4      ; have R2 set the condition codes
  BRn DivDone
  ADD R0,R0,R4
  LD R2,QUOTIENT
  ADD R2,R2,#1
  ST R2,QUOTIENT
  BR DivLoop
DivDone:  ADD R3,R3,#0         ; Negative?
  BRzp DivRet
  LD R2,QUOTIENT     ; Yes, then negate R2
  NOT R2,R2
  ADD R2,R2,#1
  ST R2,QUOTIENT
DivRet:      ST R0,REMAINDER
  LD R0, POP_IND1
  JMP R0
QUOTIENT: .FILL #0
REMAINDER: .FILL #0

.END

Assembled! Use %dis or %dump to examine; use %exe to run.


In [361]:
%bp x308B

Breakpoints
    1) ASCII_BASE: x308B: x2228  LD R1, NEG_ONE                            [line: 57]


In [362]:
%bp x3093

Breakpoints
    1) ASCII_BASE: x308B: x2228  LD R1, NEG_ONE                            [line: 57]
    2) ASCII_LOOP: x3093: x2024  LD R0, TEMPZ                              [line: 74]


In [363]:
%bp x309B

Breakpoints
    1) ASCII_BASE: x308B: x2228  LD R1, NEG_ONE                            [line: 57]
    2) ASCII_LOOP: x3093: x2024  LD R0, TEMPZ                              [line: 74]
    3)            x309B: xEC3A  LEA R6, DIVIDE                            [line: 84]


In [364]:
%bp x309E

Breakpoints
    1) ASCII_BASE: x308B: x2228  LD R1, NEG_ONE                            [line: 57]
    2) ASCII_LOOP: x3093: x2024  LD R0, TEMPZ                              [line: 74]
    3)            x309B: xEC3A  LEA R6, DIVIDE                            [line: 84]
    4)            x309E: x2459  LD R2, QUOTIENT                           [line: 87]


In [365]:
%exe

...breakpoint hit at x308B
Computation SUSPENDED
Instructions: 4
Cycles: 34 (0.000017 milliseconds)

Registers:
PC: x308B
N: 0 Z: 0 P: 1 
R0: x000C R1: x0000 R2: x0000 R3: x0000 
R4: x0000 R5: x0000 R6: x0000 R7: x0000 


In [368]:
%cont

...breakpoint hit at x309E
Computation SUSPENDED
Instructions: 136
Cycles: 1015 (0.000508 milliseconds)

Registers:
PC: x309E
N: 0 Z: 0 P: 1 
R0: x000C R1: x000A R2: x3066 R3: x0000 
R4: x0000 R5: x3002 R6: x30D6 R7: x309E 


In [93]:
%pc x3000


Registers:
PC: x3000
N: 0 Z: 1 P: 0 
R0: x0000 R1: x0000 R2: x0000 R3: x0000 
R4: x0000 R5: x0000 R6: x0000 R7: x0000 


In [309]:
%step

Stepping...  => read, <= write, (Instructions/Cycles):
    PC <= x30EE
(216/1565) ST R2, QUOTIENT [196] (x30EE*: x340A)
    memory[x30F8] <= x0010

Registers:
PC: x30EE
N: 0 Z: 0 P: 1 
R0: x0CFA R1: x000A R2: x0010 R3: x0000 
R4: xFFF6 R5: x3009 R6: x30D6 R7: x309E 


In [358]:
.ORIG x3000

ADD R0, R0, xC
ADD R1, R1, xA
LEA R6, Divide
LD R2, PUSH_IND1
JSRR R2         ; R0 / R1
LD R2, QUOTIENT

HALT

Top:    .FILL Stack
Stack:  .BLKW #100

PUSH:   LD R5, Top
        STR R0, R5, #0      ;; Save R0 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R1, R5, #0      ;; Save R1 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R2, R5, #0      ;; Save R2 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R3, R5, #0      ;; Save R3 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R4, R5, #0      ;; Save R4 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R6, R5, #0      ;; Save R6 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R7, R5, #0      ;; Save R7 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        ST R5, Top
        JMP R6
        
POP:    LD R5, Top
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R7, R5, #0      ;; Load stack into R7
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R6, R5, #0      ;; Load Stack into R6
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R4, R5, #0      ;; Load Stack into R4
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R3, R5, #0      ;; Load Stack into R3
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R2, R5, #0      ;; Load Stack into R2
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R1, R5, #0      ;; Load Stack into R1
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R0, R5, #0      ;; Load Stack into R0
        ST R5, Top
        RET
        
PUSH_IND1: .FILL PUSH
POP_IND1: .FILL POP

;;; -----------------------------------------------------
;;; Mult_Product <- R0 * R1
;;; Also uses R3 to store SIGN
Multiply: AND R2,R2,#0
  AND R3,R3,#0
  ADD R0,R0,#0         ; compare R0
  BRn MultNEG1
  BR  MultCont
MultNEG1: NOT R3,R3         ; flip SIGN
  NOT R0,R0
  ADD R0,R0,#1
MultCONT: ADD R1,R1,#0         ; compare R1
  BRn MultNEG2
  BR MultInit
MultNEG2: NOT R3,R3         ; flip SIGN
  NOT R1,R1
  ADD R1,R1,#1
MultInit: ADD R0,R0,#0      ; have R0 set the condition codes
MultLoop: BRz MultDone
  ADD R2,R2,R1
  ADD R0,R0,#-1
  BR MultLoop
MultDone: ADD R0,R3,#0
  BRzp MultRet
  NOT R2,R2
  ADD R2,R2,#1
MultRet:
  ST R2, MULT_PRODUCT           ; Product has the product
  LD R0, POP_IND1
  JMP R0
  
MULT_PRODUCT: .FILL #0

;;; -----------------------------------------------------
;;; R0 / R1
;;; Also uses R3 to store SIGN
;;;           R4 to store -R1
;;;           R5 is QUOTIENT
;;;           R6 is REMAINDER
;;;           R2 temp
Divide:   AND R3,R3,#0
  ST R3, QUOTIENT
  ST R3, REMAINDER
  ADD R0,R0,#0         ; compare R0
  BRn DivNEG1
  BR  DivCont
DivNEG1:  NOT R3,R3         ; flip SIGN
  NOT R0,R0
  ADD R0,R0,#1
DivCONT:  ADD R1,R1,#0         ; compare R1
  BRn DivNEG2
  BR DivInit
DivNEG2:  NOT R3,R3         ; flip SIGN
  NOT R1,R1
  ADD R1,R1,#1
DivInit:  ADD R4,R1,#0
  NOT R4,R4
  ADD R4,R4,#1
DivLoop:  ADD R2,R0,R4      ; have R2 set the condition codes
  BRn DivDone
  ADD R0,R0,R4
  LD R2,QUOTIENT
  ADD R2,R2,#1
  ST R2,QUOTIENT
  BR DivLoop
DivDone:  ADD R3,R3,#0         ; Negative?
  BRzp DivRet
  LD R2,QUOTIENT     ; Yes, then negate R2
  NOT R2,R2
  ADD R2,R2,#1
  ST R2,QUOTIENT
DivRet:      ST R0,REMAINDER
  LD R0, POP_IND1
  JMP R0
QUOTIENT: .FILL #0
REMAINDER: .FILL #0        
        
.END

Assembled! Use %dis or %dump to examine; use %exe to run.


In [359]:
%exe

Computation completed
Instructions: 67
Cycles: 497 (0.000249 milliseconds)

Registers:
PC: x048E
N: 0 Z: 0 P: 1 
R0: x000C R1: x000A R2: x0001 R3: x0000 
R4: x0000 R5: x3008 R6: x30AB R7: x3007 


In [355]:
%dis x3064

Memory disassembled:
           x3064: x0000 - \0
           x3065: x0000 - \0
PUSH:      x3066: x2B9A  LD R5, TOP                                [line: 9]
           x3067: x7140  STR R0, R5, 0                             [line: 10]
           x3068: x1B61  ADD R5, R5, #1                            [line: 11]
           x3069: x7340  STR R1, R5, 0                             [line: 12]
           x306A: x1B61  ADD R5, R5, #1                            [line: 13]
           x306B: x7540  STR R2, R5, 0                             [line: 14]
           x306C: x1B61  ADD R5, R5, #1                            [line: 15]
           x306D: x7740  STR R3, R5, 0                             [line: 16]
           x306E: x1B61  ADD R5, R5, #1                            [line: 17]
           x306F: x7940  STR R4, R5, 0                             [line: 18]
           x3070: x1B61  ADD R5, R5, #1                            [line: 19]
           x3071: x7D40  STR R6, R5, 0                          